In [7]:
import requests

# Define the URL and the payload
url = 'http://localhost:11434/api/generate'
payload = {
    "model": "llama3",
    "prompt": "Why is the sky blue?",
    "stream": False
}

# Make a POST request
response = requests.post(url, json=payload)

# Print the response text
print(response.json()['response'])

What a great question!

The sky appears blue because of a phenomenon called scattering. When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths.

This is known as Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described the phenomenon in the late 19th century. The shorter blue wavelengths are scattered in every direction and reach our eyes from all parts of the sky, making it appear blue.

Here's a simplified explanation:

1. Sunlight enters the Earth's atmosphere.
2. The light encounters tiny molecules of gases (N2 and O2).
3. These molecules scatter the light in all directions.
4. Shorter wavelengths (blue) are scattered more than longer wavelengths (red).
5. Our eyes perceive this scattered blue light as the color we see in the sky.

Other factors can affect t

In [1]:
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model='llama3')

loader = PyPDFLoader('Thesis_Premise.pdf')
document_contents = loader.load_and_split()

model = Ollama(model = 'llama3',num_ctx = 9000,base_url ='http://localhost:11434') # change with other  /api/generate',base_url ='http://localhost:11434'

chat_history = []

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            'You are an AI named okaygpt and you are working for Mercedes Benz Buses and Truck'
        ),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human','{input}'),
    ]
)


chain = prompt_template | model

def start_app():
    message_count = 0
    max_message = 3
    while True and message_count <= max_message:
        question  = input('You: ')
        print()
        print(f'User : {question}')
        print()
        if question == 'done':
            return 
        
        response = chain.invoke({'input':question,'chat_history':chat_history})
        chat_history.append(HumanMessage(content=question))
        chat_history.append(AIMessage(content = response))
        print('AI:' + response )
        message_count += 1
    print('AI TERMINATED ITSELF!')





## PDF LOADER

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

loader = PyPDFLoader('Thesis_Premise.pdf')
document_contents = loader.load_and_split()
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'Thesis_Premise.pdf', 'page': 0}, page_content='Industrial Engineering Design  \nProject Information Form  \nProject Title  : Price Forecasting in Energy Markets  \nGroup Members  : Emre Bozok, Ahmet Şeref Okay  \nAdvisor  : Hüseyin Kutay Tinç  \nCourse Code  : ☐  END4901 ☒ END4902  \nEngineering Design Field  : ☒ Product/Service ☐ Process  \nDesign project topic and scope : (Identification of the system and its components where the design project will be carried out, \nexplaining where to focus within the scope of the project)  \nEnergy markets are the markets that electricity is the main asset that  is bought or sold and it cannot be stored. \nWhen it is generated, it should be consumed immediately. Also, the amount of electricity generated and \nconsumed must be equal to each other. It is in a balance. There are three energy markets in Turkey, day \nahead , intra -day, and balancing market. In the day ahead market, supply and demand information is obtai

In [3]:
template = """
Answer the question based on the context based below. If you can't 
answer the question, reply 'Sorry, I don't know'.

Context: {context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)
prompt.format(context= 'Here is some context', question = "Here is a question")



"\nAnswer the question based on the context based below. If you can't \nanswer the question, reply 'Sorry, I don't know'.\n\nContext: Here is some context\n\nQuestion: Here is a question\n"

In [4]:
chain2 = prompt | model | parser

In [10]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [11]:
retriever = vectorstore.as_retriever()
retriever.invoke("what are the Methods of Evaluation? like simulation and testing:")

[Document(metadata={'source': 'Thesis_Premise.pdf', 'page': 4}, page_content="Industrial Engineering Design  \nProject Information Form  \nACM/IEEE International Symposium on Empirical Software Engineering and Measurement (ESEM) , Porto \nde Galinhas, Brazil, 2019, pp. 1 -6, doi: 10.1109/ESEM.2019.8870157.  \n3. Strategy Planning:  \nUse: Strategy planning is crucial in the initial stages of the project for defining objectives, identifying resources, \nand setting timelines. It is also vital in later stages for adapting to changes and evaluating progress.  \nJustification : Strategic planning in industrial engineering involves setting goals, analyzing the competitive \nenvironment, and assessing internal organizational capabilities. It is crucial for aligning the project's objectives \nwith market needs and organizational capabil ities. For our project Strategy Planning ensures that our project is \nremains focused and adaptable.  \nEvidence : In the context of energy markets, a study 

In [12]:
from operator import itemgetter

chain2 = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [76]:
question  = 'Can you list me Methods of Producing Alternatives'

In [77]:
print(chain2.invoke({'question': question}))

Based on the context provided, the methods of producing alternatives are:

1. Literature Review and Benchmarking: Researching existing solutions and best practices in the field to understand what has been successful and why.
2. Brainstorming Sessions: Engaging with experts, stakeholders, and peers in brainstorming sessions to generate creative and diverse solutions.
3. Technological Feasibility Studies: Assessing new and emerging technologies to understand their potential application in your project.

Please note that these are the methods mentioned in the provided context. If there is more information available or if you would like me to clarify any of these points, please let me know!


In [88]:
loader = PyPDFLoader('Thesis_Premise.pdf')
pages = loader.load_and_split()

# Initialize embeddings and model
embeddings = OllamaEmbeddings(model='llama3')
model = Ollama(model='llama3', num_ctx=9000, base_url='http://localhost:11434')
parser = StrOutputParser()

# Create Vector Store from documents
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

# Define the RAG Prompt Template
template = """
Answer the question based on the context below. If you can't 
answer the question, reply 'Sorry, I don't know'.

Context: {context}

Question: {question}
"""
rag_prompt = PromptTemplate.from_template(template)

# Merge RAG with the chatbot
prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are an AI named okaygpt and you are working for Mercedes Benz Buses and Truck'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human', '{input}'),
    ]
)

def start_app():
    message_count = 0
    max_message = 3
    while message_count <= max_message:
        question = input('You: ')
        print(f'User: {question}\n')

        if question.lower() == 'done':
            print('Session ended.')
            break
        
        # Retrieve context from documents
        context = retriever.invoke(question)

        # Format the RAG prompt
        formatted_prompt = rag_prompt.format(context=context, question=question)

        # Get response from the chain
        response = (formatted_prompt | model | parser).invoke()
        
        chat_history.append(HumanMessage(content=question))
        chat_history.append(AIMessage(content=response))
        
        print('AI:', response)
        message_count += 1
    
    print('AI terminated itself!')



In [ ]:
'Can you list me Methods of Producing Alternatives'

In [6]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.llms import Ollama
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [3]:
loader = PyPDFLoader('Thesis_Premise.pdf')
pages = loader.load_and_split()

# Initialize embeddings and model
embeddings = OllamaEmbeddings(model='llama3')
model = Ollama(model='llama3', num_ctx=9000, base_url='http://localhost:11434')
parser = StrOutputParser()
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
# Create Vector Store from documents
vectorstore = DocArrayInMemorySearch.from_documents(all_splits, embedding=embeddings)
# DocArrayInMemorySearch.from_documents(all_splits, embedding=embeddings)
# Create a retriever from the vector store
retriever = vectorstore.as_retriever()


c:\Users\ahmet\Desktop\AI-TRIES\venv\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [9]:
template = """
You are an AI named okaygpt and you are working for Mercedes Benz Buses and Truck.
Answer the question based on the context below and the chat history. If you can't 
answer the question based on the given information, reply 'Sorry, I don't know'.

Context: {context}

Chat History: {chat_history}

Question: {question}

Answer:
"""

rag_prompt = PromptTemplate.from_template(template)

# Initialize conversation memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": rag_prompt}
)

def start_app():
    message_count = 0
    max_message = 3
    while message_count <= max_message:
        question = input('You: ')
        print(f'User: {question}\n')

        if question.lower() == 'done':
            print('Session ended.')
            break
        
        # Get response from the chain
        response = qa_chain({"question": question})
        
        print('AI:', response['answer'])
        message_count += 1
    
    print('AI terminated itself!')

In [10]:
start_app()

User: can you explain this : The needs that will be met by the design and the requirements of the design

AI: As okaygpt, I'd be happy to help explain!

The design project aims to provide forecasts about the upcoming movements in the energy markets for the stakeholders (power plants, wholesale companies, and retail companies). This forecast-based insight will enable these players to optimize their market positions, react proactively to potential market shifts, and make informed decisions about buying, selling, or waiting for the right times.

The needs that will be met by this design include:

* Meeting the need for timely and accurate information about energy market trends, allowing stakeholders to make informed decisions.
* Providing a competitive advantage by offering insights into future market movements, enabling stakeholders to optimize their strategies and maximize profits or minimize losses.
* Supporting the development of short-term and long-term strategies for energy market p